<a href="https://colab.research.google.com/github/fubotz/ICL_2024W/blob/main/FinalProject_Fabian_SCHAMBECK_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ICL Final Project: Finetuning a Pretrained Multilingual Model for Cognate Detection

Model: distilbert-base-multilingual-cased

Dataset: custom dataset containing en-fr cognates (Frossard et al.)

Method: [MASK] approach

In [63]:
!pip install bertviz
!pip install datasets
!pip install evaluate
!pip install scikit-learn
!pip install transformers
!pip install torch

In [64]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Load Dataset ##

In [65]:
!wget https://raw.githubusercontent.com/fubotz/ICL_2024W/refs/heads/main/word_pairs.json        # dataset taken from Frossard et al.

--2025-01-27 22:38:13--  https://raw.githubusercontent.com/fubotz/ICL_2024W/refs/heads/main/word_pairs.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23242 (23K) [text/plain]
Saving to: ‘word_pairs.json.2’

word_pairs.json.2   100%[===================>]  22.70K  --.-KB/s    in 0s      

2025-01-27 22:38:13 (146 MB/s) - ‘word_pairs.json.2’ saved [23242/23242]



In [66]:
import json
with open("word_pairs.json", "r") as f:
    dataset = json.load(f)
print(dataset)

[{'abandon': 'abandon'}, {'abbe': 'abbé'}, {'abdomen': 'abdomen'}, {'abdominal': 'abdominal'}, {'aberration': 'aberration'}, {'abolition': 'abolition'}, {'abominable': 'abominable'}, {'absence': 'absence'}, {'absolute': 'absolu'}, {'absolution': 'absolution'}, {'absorption': 'absorption'}, {'abstinence': 'abstinence'}, {'abstraction': 'abstraction'}, {'absurd': 'absurde'}, {'absurdity': 'absurdité'}, {'abundance': 'abondance'}, {'abundant': 'abondant'}, {'academic': 'académique'}, {'academy': 'académie'}, {'acceleration': 'accélération'}, {'accent': 'accent'}, {'acceptable': 'acceptable'}, {'access': 'accès'}, {'accessory': 'accessoire'}, {'accident': 'accident'}, {'accidental': 'accidentel'}, {'accidentally': 'accidentellement'}, {'acclamation': 'acclamation'}, {'accord': 'accord'}, {'acetone': 'acétone'}, {'acid': 'acide'}, {'acoustic': 'acoustique'}, {'activity': 'activité'}, {'actor': 'acteur'}, {'addition': 'addition'}, {'address': 'addresse'}, {'adherent': 'adhérent'}, {'administ

In [67]:
import random

# Shuffle the dataset
random.shuffle(dataset)

# Calculate the split indices
total_size = len(dataset)
train_size = int(0.7 * total_size)
val_size = int(0.2 * total_size)

# Create train, validation, and test splits (70:20:10)
train_data = dataset[:train_size]
val_data = dataset[train_size:train_size + val_size]
test_data = dataset[train_size + val_size:]

# Verify the split sizes
print(f"Total samples: {total_size}")
print(f"Training samples: {len(train_data)}")
print(f"Validation samples: {len(val_data)}")
print(f"Test samples: {len(test_data)}")

Total samples: 492
Training samples: 344
Validation samples: 98
Test samples: 50


## Load Model ##

In [68]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

# Load tokenizer and models
model_name = "distilbert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Model for [MASK] Approach
pretrained_model = AutoModelForMaskedLM.from_pretrained(model_name)

print(tokenizer)

DistilBertTokenizerFast(name_or_path='distilbert-base-multilingual-cased', vocab_size=119547, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)


## Preprocessing for [MASK] Evaluation ##

In [69]:
import torch

# Define the evaluation function
def evaluate_mask_accuracy(model, test_data, tokenizer, top_k=5):
    """
    Evaluates the accuracy of a masked language model on a cognate dataset.

    Args:
        model: The pretrained or fine-tuned masked language model.
        test_data (list of dict): Dataset with English-French cognate pairs.
        tokenizer: The tokenizer corresponding to the model.
        top_k (int): Number of top predictions to consider for accuracy.

    Returns:
        float: Accuracy of the model on the dataset.
    """
    correct_predictions = 0
    total_samples = len(test_data)

    print("Evaluating MASK predictions...\n")
    for i, pair in enumerate(test_data):
        # Extract the English word and expected French cognate
        english_word, french_word = list(pair.items())[0]

        # Construct the masked input sentence
        sentence = f"The English word is: {english_word}. Le mot français est: [MASK]."
        inputs = tokenizer(sentence, return_tensors="pt")
        mask_token_index = (inputs.input_ids == tokenizer.mask_token_id).nonzero(as_tuple=True)[1]

        # Forward pass through the model
        outputs = model(**inputs)
        logits = outputs.logits

        # Get top-k predictions for the [MASK] token
        mask_token_logits = logits[0, mask_token_index, :]
        top_k_tokens = torch.topk(mask_token_logits, k=top_k, dim=1).indices[0].tolist()
        predicted_words = [tokenizer.decode([token]).strip() for token in top_k_tokens]

        # Log predictions
        print(f"Instance {i+1}:")
        print(f"  English word: {english_word}")
        print(f"  Expected French word: {french_word}")
        print(f"  Predicted MASK words: {predicted_words}\n")

        # Check if the expected French word is in the predictions
        if french_word in predicted_words:
            correct_predictions += 1

    # Compute accuracy
    accuracy = correct_predictions / total_samples
    return accuracy

# Evaluate the accuracy
accuracy = evaluate_mask_accuracy(pretrained_model, test_data, tokenizer, top_k=5)
print(f"Accuracy of the model: {accuracy:.2%}")

Evaluating MASK predictions...

Instance 1:
  English word: accidentally
  Expected French word: accidentellement
  Predicted MASK words: ['loin', 'refuge', 'très', 'fruit', 'rose']

Instance 2:
  English word: acclamation
  Expected French word: acclamation
  Predicted MASK words: ['langue', 'accent', 'enfant', 'action', 'homme']

Instance 3:
  English word: adventure
  Expected French word: aventure
  Predicted MASK words: ['俺', 'voyage', 'action', 'fortune', 'mission']

Instance 4:
  English word: music
  Expected French word: musique
  Predicted MASK words: ['musique', 'music', 'composer', 'chant', 'sound']

Instance 5:
  English word: longitude
  Expected French word: longitude
  Predicted MASK words: ['latitude', 'hauteur', 'longitud', 'longueur', 'cercle']

Instance 6:
  English word: circumstance
  Expected French word: circonstance
  Predicted MASK words: ['cercle', 'confusion', 'esprit', '诵', 'conscience']

Instance 7:
  English word: quest
  Expected French word: quête
  Pre

## Preprocess Dataset ##

In [70]:
from torch.utils.data import DataLoader

# Preprocessing function
def preprocess_function(examples):
    """
    Preprocess examples by tokenizing English and French text and preparing input and label tensors.
    """
    # Extract inputs and targets
    inputs = examples["word_en"]        # List of English words
    targets = examples["word_fr"]       # List of French words

    # Tokenize inputs and targets with a smaller max_length
    model_inputs = tokenizer(inputs, max_length=8, truncation=True, padding="max_length")
    labels = tokenizer(targets, max_length=8, truncation=True, padding="max_length")["input_ids"]

    # Replace padding tokens in labels with -100
    model_inputs["labels"] = [
        [-100 if token == tokenizer.pad_token_id else token for token in seq] for seq in labels
    ]

    return model_inputs

In [71]:
from datasets import Dataset

# Convert raw data into the correct format
formatted_train_data = [{"word_en": k, "word_fr": v} for item in train_data for k, v in item.items()]
formatted_val_data = [{"word_en": k, "word_fr": v} for item in val_data for k, v in item.items()]
formatted_test_data = [{"word_en": k, "word_fr": v} for item in test_data for k, v in item.items()]

# Create Hugging Face datasets
train_dataset = Dataset.from_list(formatted_train_data)
val_dataset = Dataset.from_list(formatted_val_data)
test_dataset = Dataset.from_list(formatted_test_data)

In [72]:
# Apply preprocessing
train_dataset = train_dataset.map(preprocess_function, batched=True)
val_dataset = val_dataset.map(preprocess_function, batched=True)
test_dataset = test_dataset.map(preprocess_function, batched=True)

# Set format for PyTorch
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
val_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

Map:   0%|          | 0/344 [00:00<?, ? examples/s]

Map:   0%|          | 0/98 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

In [73]:
# Inspect a preprocessed example from the training set
print("\nSample preprocessed training example:")
sample_train_example = train_dataset[0]  # Access the first example in train_dataset
print(sample_train_example)

# Decode the input IDs and labels to verify correctness
decoded_input = tokenizer.decode(sample_train_example["input_ids"].tolist(), skip_special_tokens=True)
decoded_label = tokenizer.decode(
    [token for token in sample_train_example["labels"].tolist() if token != -100], skip_special_tokens=True
)

print("\nDecoded Training Example:")
print(f"Input (word_en): {decoded_input}")
print(f"Label (word_fr): {decoded_label}")

# Verify dataset sizes after preprocessing
print(f"\nFinal dataset sizes:")
print(f"Training set: {len(train_dataset)}")
print(f"Validation set: {len(val_dataset)}")
print(f"Test set: {len(test_dataset)}")


Sample preprocessed training example:
{'input_ids': tensor([  101, 29580,   102,     0,     0,     0,     0,     0]), 'attention_mask': tensor([1, 1, 1, 0, 0, 0, 0, 0]), 'labels': tensor([  101, 12604, 51584,   102,  -100,  -100,  -100,  -100])}

Decoded Training Example:
Input (word_en): ideal
Label (word_fr): idéal

Final dataset sizes:
Training set: 344
Validation set: 98
Test set: 50


## Finetune Model ##

In [74]:
from transformers import TrainingArguments
import evaluate
import numpy as np

# Load accuracy metric
accuracy = evaluate.load("accuracy")

# Define metric computation function
def compute_metrics(eval_pred):
    """
    Computes accuracy during validation by ignoring padding tokens.
    """
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    # Flatten predictions and labels (remove -100 labels)
    flattened_predictions = []
    flattened_labels = []

    for pred, label in zip(predictions, labels):
        for p, l in zip(pred, label):
            if l != -100:  # Ignore padding token labels
                flattened_predictions.append(p)
                flattened_labels.append(l)

    return accuracy.compute(predictions=flattened_predictions, references=flattened_labels)

In [75]:
from transformers import Trainer

# Define training arguments
arguments = TrainingArguments(
    output_dir="/content/drive/MyDrive/Colab Notebooks/cognate_trainer",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    logging_steps=8,
    num_train_epochs=8,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=1e-5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    report_to='none',
    seed=224
)

# Initialize the Trainer
trainer = Trainer(
    model=pretrained_model,
    args=arguments,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,       # NB: change for test
    processing_class=tokenizer,
    compute_metrics=compute_metrics,
)

In [76]:
# Verify dataset format
from torch.utils.data import DataLoader

# Create a DataLoader for debugging
debug_loader = DataLoader(train_dataset, batch_size=8)

# Get a batch
batch = next(iter(debug_loader))
print(batch)

# Print shapes
print(f"Input IDs shape: {batch['input_ids'].shape}")
print(f"Attention Mask shape: {batch['attention_mask'].shape}")
print(f"Labels shape: {batch['labels'].shape}")

{'input_ids': tensor([[  101, 29580,   102,     0,     0,     0,     0,     0],
        [  101, 14379, 76299,   102,     0,     0,     0,     0],
        [  101, 46176,   102,     0,     0,     0,     0,     0],
        [  101, 12533,   102,     0,     0,     0,     0,     0],
        [  101, 49523, 14786, 13275,   102,     0,     0,     0],
        [  101, 38676, 12717, 48532, 10415,   102,     0,     0],
        [  101, 11487,   102,     0,     0,     0,     0,     0],
        [  101, 20455,   102,     0,     0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 0, 0, 0, 0],
        [1, 1, 1, 0, 0, 0, 0, 0],
        [1, 1, 1, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 0, 0, 0, 0, 0],
        [1, 1, 1, 0, 0, 0, 0, 0]]), 'labels': tensor([[  101, 12604, 51584,   102,  -100,  -100,  -100,  -100],
        [  101, 14379, 45375,   102,  -100,  -100,  -100,  -100],
        [  101, 13621,

In [77]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,4.504300,3.318940,0.382812
2,1.961000,1.970660,0.726562
3,1.615200,1.849468,0.739583
4,1.270000,1.777911,0.742188
5,0.877400,1.779011,0.736979
6,0.812100,1.789871,0.736979
7,0.937500,1.802764,0.739583
8,0.628700,1.801697,0.739583


There were missing keys in the checkpoint model loaded: ['vocab_projector.weight'].


TrainOutput(global_step=344, training_loss=1.9857327175694843, metrics={'train_runtime': 439.0423, 'train_samples_per_second': 6.268, 'train_steps_per_second': 0.784, 'total_flos': 5711889558528.0, 'train_loss': 1.9857327175694843, 'epoch': 8.0})

In [78]:
# Save the trained model
output_dir = "/content/drive/MyDrive/Colab Notebooks/cognate_trainer_best_model"
trainer.save_model(output_dir)

# Evaluate the model on the test dataset
test_results = trainer.evaluate(test_dataset)

print("\nTest Results:")
print(test_results)


Test Results:
{'eval_loss': 1.0440846681594849, 'eval_accuracy': 0.8305084745762712, 'eval_runtime': 1.569, 'eval_samples_per_second': 31.868, 'eval_steps_per_second': 4.461, 'epoch': 8.0}


In [79]:
finetuned_model = AutoModelForMaskedLM.from_pretrained(output_dir)

## Evaluate Finetuned Model ##

In [81]:
# Evaluate the accuracy
accuracy = evaluate_mask_accuracy(finetuned_model, test_data, tokenizer, top_k=5)
print(f"Accuracy of the model: {accuracy:.2%}")

Evaluating MASK predictions...

Instance 1:
  English word: accidentally
  Expected French word: accidentellement
  Predicted MASK words: ['feu', 'vert', '歙', 'ँ', '嘯']

Instance 2:
  English word: acclamation
  Expected French word: acclamation
  Predicted MASK words: ['ँ', 'langue', 'fer', 'ouverture', 'emploi']

Instance 3:
  English word: adventure
  Expected French word: aventure
  Predicted MASK words: ['voyage', '뢰', 'dire', 'feu', 'vert']

Instance 4:
  English word: music
  Expected French word: musique
  Predicted MASK words: ['musique', 'voix', 'danse', 'orchestre', 'esprit']

Instance 5:
  English word: longitude
  Expected French word: longitude
  Predicted MASK words: ['latitude', 'cercle', 'hauteur', 'longueur', 'longue']

Instance 6:
  English word: circumstance
  Expected French word: circonstance
  Predicted MASK words: ['commun', 'cercle', '嘯', '₤', 'vert']

Instance 7:
  English word: quest
  Expected French word: quête
  Predicted MASK words: ['passion', 'ً', 'dire

## Visualization ##

In [82]:
# visualization